# GPUs are a must for data science
GPUs increase performance of data science activities significantly. This notebook checks if your GPU is properly configured and available to be leveraged.

- [Read TensorFlow docs here](https://www.tensorflow.org/api_docs/python/tf/all_symbols)
- [Tensor GPU Docs](https://www.tensorflow.org/guide/gpu)

Note: as per TensorFlow docs "TensorFlow code, and tf.keras models will transparently run on a single GPU with no code changes required. The simplest way to run on multiple GPUs, on one or many machines, is using Distribution Strategies."

## Check the number of GPUs

In [5]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
print("Physical GPUs count", len(gpus))

Physical GPUs count 0


## See what else TensorFlow can tell us

In [8]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13739859749333745835]

## Display details using NVIDIA System Management Interface

In [20]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


## OK what about CPU and memory

In [19]:
!cat /proc/cpuinfo
!cat /proc/meminfo

cat: /proc/cpuinfo: No such file or directory
cat: /proc/meminfo: No such file or directory


## Let's try to do some Maths

In [17]:
tf.debugging.set_log_device_placement(True)

# Create some tensors - should use the GPU if available
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [20]:
tf.debugging.set_log_device_placement(True)

# Place tensors on the CPU
with tf.device('/CPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# Run on the GPU
c = tf.matmul(a, b)
print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


## Try distributing across the GPUs - if you have more than one

In [19]:
tf.debugging.set_log_device_placement(True)

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  inputs = tf.keras.layers.Input(shape=(1,))
  predictions = tf.keras.layers.Dense(1)(inputs)
  model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
  model.compile(loss='mse',
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.2))
    

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
